# A simple demondstration of Bayesian Networks

In this notebook, I'm going to use the [Julia](https://julialang.org/) programming language to demonstrate some of the basic ideas of a Bayesian network. We will use the Julia package [BayesNets.jl](https://sisl.github.io/BayesNets.jl/dev/) to build and reason with a Bayesian network

## Main Assumption

- All our variables are discrete (take a finite number of possible values)

In [ ]:
using BayesNets

In [ ]:
using TikzGraphs # required to plot tex-formatted graphs (recommended), otherwise GraphPlot.jl is used


## Example



I'm looking up at the sky and I see something. Is it a bird or a plane or Superman? How happy I am depends on what I see. I can see one of three things:

1. A bird
1. A plane
1. Superman

The Bayesian network will look like the following:

![superman](./superman.svg)

For nodes that don't have any parents (no edges coming into them), we provide the prior probabilities (_a priori_). There are three states for `:seeing` so I need to provide three prior probabilities

What I see (or don't see) has to be exhaustive and the probabilities for seeing a bird, a plane, or Superman have to add up to 1

1. $P(seeing=\text{A bird (state 1)})=0.40$
1. $P(seeing=\text{A plane (state 2)})=0.55$
1. $P(seeing=\text{Superman (state 3)})=1-(0.40+0.55)=0.05$

`:happy` has parents so for happy I need to provide the conditional probabilities for each state of `:happy` ($\text{:happy}_i$) for each state of `:seeing` ($\text{:seeing}_j$): $P(\text{:happy}_i|\text{:seeing}_j)$. Namely

| `:seeing`  | `:happy`      | P(`:seeing`,`:happy`) |
|:-----------|:--------------|:------:|
|`:bird`     | -`:happy`     | 0.4|
|`:plane`    | -`:happy`    | 0.85|
|`:superman` | -`:happy`    | 0.01 |
|`:bird`     | +`:happy`        |0.6 |
|`:plane`    | +`:happy`     |0.15 |
|`:superman` | +`:happy`    | 0.99 |


This is _factor table_ is in the order of variables expected in `BayesNets.jl`.

In [ ]:
bn2a = DiscreteBayesNet()
push!(bn2a, DiscreteCPD(:sighted, [0.40, 0.55, 0.05])) # Prior probabilities for seeing states
push!(bn2a, CategoricalCPD(:happy, [:sighted], [3], [Categorical([0.4,0.6]), # (Prob not happy | bird, Prob happy| bird) sums to 1
                                                     Categorical([0.85,0.15]), # (Prob not happy | plane, Prob happy | bird) sums to 1
                                                     Categorical([0.01,0.99]),])) # (Prob not happy | Superman, Prob happy | Superman) sums to 1

### We can check the generated factor table to make sure it is what we expected

- happy=1 means not happy
- happy=2 means happy

In [ ]:
table(bn2a, :happy)

## Inferencing 

Now that we have our model, we can do inferencing

We start off by calculating what the probabilities of being not happy (1) or happy (2) without any evidence

In [ ]:
f1 = infer(bn2a, :happy)

### Check

The probability of being happy is just the sum of all the ways we can be happy:

Probability of seeing bird __times__ the probability of being happy seeing a bird __plus__ probability of seeing plan __times__ the probability of being happy seeing a plane __plus__ probability of seeing Superman __times__ the probability of being happy seeing Superman

In [ ]:
0.4*0.6+0.55*0.15+0.05*0.99

## We can also verify that the probility of happy and not happy sum to 1

In [ ]:
sum(f1.potential)

### What is the probability of seeing a plane given that I'm happy?

In [ ]:
f2 = infer(bn2a, :sighted, evidence=Assignment(:happy=>2))

In [ ]:
sum(f2.potential)

## Example: Satellite Monitoring

Here is an example from _Algorithms for Decision Making_

- `B` battery failure
- `S` solar panel failure
- `E` electrical system failure D trajectory deviation
- `C` communication loss

>Fortunately, battery failure and solar panel failures are both rare, although solar panel failures are somewhat more likely than battery failures. Failures in either can lead to an electrical system failure. There may be causes of electrical system failure other than battery or solar panel failure, such as a problem with the power management unit. An electrical system failure can result in trajectory deviation, which can be observed from the earth by telescope, as well as a communication loss that interrupts the transmission of telemetry and mission data down to various ground stations. Other anomalies not involving the electrical system can result in trajectory deviation and communication loss. (p. 34)

The Bayesian network for this model looks like this:

<img src="sat.png" alt="satellite model" width="175">


In [ ]:
bns = DiscreteBayesNet()
push!(bns, DiscreteCPD(:B, [0.99, 0.01]))
push!(bns, DiscreteCPD(:S, [0.98, 0.02]))
push!(bns, DiscreteCPD(:E, [:B, :S], [2,2],
        [Categorical([0.9, 0.1]), # (B=1 & S=1 & E=1, B=1 & S=1 & S=2)
         Categorical([0.05,0.95]), #(B=2 & S=1 & E=1, B=2 & S=1 & E=2)
         Categorical([0.04,0.96]), #(B=1 & S=2 & E=1, B=1 & S=2 & E=2)
         Categorical([0.01,0.99])])) #(B=2 & S=2 & E=1, B=2 & S=2 & E=2)
push!(bns, DiscreteCPD(:D, [:E], [2],
        [Categorical([0.96, 0.04]), #(E=1 & D=1, E=1 & D=2)
         Categorical([0.03, 0.97])])) #(E=2 & D=1, E=2 & D=2)
push!(bns, DiscreteCPD(:C, [:E], [2],
        [Categorical([0.98, 0.02]), #(E=1 & C=1, E=1 & C=2)
         Categorical([0.01, 0.99])])) #(E=2 & D=1, E=2 & D=2)


In [ ]:
table(bns, :E)

### What is the probability of there being a battery failure given that we have a trajectory deviation?

- We specify what we know using `evidence` and `Assignment`
  - In this case we are assigning the value of 2 (True) to `:D` (deviation)
- We then `infer` the value we are interested, in this case `:B` (battery)

In [ ]:
infer(bns, :B, evidence=Assignment(:D=>2))

### What is the state of the battery if we have trajectory deviation and we know the solar panel has failed?

In [ ]:
infer(bns, :B, evidence=Assignment(:D=>2, :S=>2))

## Example: COVID, Smoking, and Hospitalization

This is a modification of a network shared [here](https://github.com/sisl/BayesNets.jl/issues/89#issue-623287862).

<img src="covid.png" alt="covid model" width="175">

- 25% of the community smokes
- 10% of the community has COVID-19
- 1% of non-smokers, COVID-19 negative people are hospitalized
- 5% of smokers, COVID-19 negative people are hospitalized
- 10% of non-smokers, COVID-19 positive people are hospitalized
- 20% of smokers, COVID-19 positive people are hospitalized

In [ ]:
bnc = DiscreteBayesNet()
push!(bnc, DiscreteCPD(:Smoker, [0.75,0.25]))
push!(bnc, DiscreteCPD(:COVID19, [0.9,0.1]))
push!(bnc, DiscreteCPD(:Hospitalized, [:Smoker, :COVID19], [2,2], 
        [Categorical([0.99,0.01]), # (Prob -hosp given -smoke AND -covid,Prob +hosp given -smoke AND -covid)
         Categorical([0.95,0.05]), # (Prob -hosp given +smoke AND -covid, Prob +hosp given +smoke AND -covid)
         Categorical([0.90,0.10]), # (Prob -hosp given -smoke AND +covid, Prob +hosp given -smoke AND +covid)
         Categorical([0.60,0.40]), # (Prob -hosp given +smoke AND +covid, Prob +hosp given +smoke AND +covid)
        ]))

In [ ]:
table(bnc, :Hospitalized)

### What are our probabilities of being hospitalized knowing nothing about the patient?

In [ ]:
f3 = infer(bnc, :Hospitalized)

In [ ]:
sum(f3.potential)

### What are our probabilities of being hospitalized knowing +covid?

In [ ]:
f3 = infer(bnc, :Hospitalized, evidence=Assignment(:COVID19=>2))

### Knowing nothing other than the person is hospitalized, what are the chances they have COVID?

In [ ]:
infer(bnc, :COVID19, evidence=Assignment(:Hospitalized=>2))

In [ ]:
infer(bnc, :COVID19, evidence=Assignment(:Hospitalized=>2, :Smoker=>2))

### Change the values around and see how the behavior changes
- Remember the values in the square brackets have to add up to 1
- If they don't, you'll get an error like this:

```
DomainError with [0.8, 0.4]:
Categorical: vector p is not a probability vector

Stacktrace:
 [1] #117
   @ ~/.julia/packages/Distributions/1PkiH/src/univariate/discrete/categorical.jl:30 [inlined]
 [2] check_args
   @ ~/.julia/packages/Distributions/1PkiH/src/utils.jl:89 [inlined]
 [3] #_#116
   @ ~/.julia/packages/Distributions/1PkiH/src/univariate/discrete/categorical.jl:30 [inlined]
 [4] #Categorical#119
   @ ~/.julia/packages/Distributions/1PkiH/src/univariate/discrete/categorical.jl:34 [inlined]
 [5] (Categorical{P} where P<:Real)(p::Vector{Float64})
   @ Distributions ~/.julia/packages/Distributions/1PkiH/src/univariate/discrete/categorical.jl:34
 [6] top-level scope
   @ In[190]:4
 [7] eval
   @ ./boot.jl:373 [inlined]
 [8] include_string(mapexpr::typeof(REPL.softscope), mod::Module, code::String, filename::String)
   @ Base ./loading.jl:1196
```
Just fix the values and rerun.